<a href="https://colab.research.google.com/github/catastropiyush/RAG-dataset-gen/blob/main/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

In [2]:
#load the LLM
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [4]:
#Example
text_input    = "A new quaternary magnesium based alloy (La0.65Ca0.35)(Mg1.32Ca0.68)Ni9 and its hydride have been synthesized and their crystal structures were determined by Guinier–Hägg X-ray powder diffraction. The compound has a hexagonal structure and is isostructural with LaMg2Ni9 (AB2C9 type), in which Ca partially occupies both A and B sites. The hydrogen absorption/desorption properties were determined by pressure–composition isotherms and compared with LaMg2Ni9. (La0.65Ca0.35)(Mg1.32Ca0.68)Ni9 absorbs ∼1.87 wt.% H2 at ∼3.3 MPa H2 and 283 K."
text_instruct = """
  Describe all the parameters of the material discussed in the text.
  If no information is available just write "N/A".
  The output should be concise and in the format as below:

  Name of Alloy                        :
  Hydrogen storage capacity            :
  Temperature                          :
  Pressure                             :
  Experimental Conditions              :
  """

In [5]:
#Example inference
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        text_instruct, # instruction
        text_input,    # input
        "",            # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:

  Describe all the parameters of the material discussed in the text.
  If no information is available just write "N/A".
  The output should be concise and in the format as below:

  Name of Alloy                        :
  Hydrogen storage capacity            :
  Temperature                          :
  Pressure                             :
  Experimental Conditions              :
  

### Input:
A new quaternary magnesium based alloy (La0.65Ca0.35)(Mg1.32Ca0.68)Ni9 and its hydride have been synthesized and their crystal structures were determined by Guinier–Hägg X-ray powder diffraction. The compound has a hexagonal structure and is isostructural with LaMg2Ni9 (AB2C9 type), in which Ca partially occupies both A and B sites. The hydrogen absorption/desorption properties were determined by pressu

#Dataset

In [ ]:
#load a csv/xls file containing the different abstracts
import pandas as pd
df = pd.read_excel('savedrecs(33).xls')

In [ ]:
selected_df = df.sample(30)
selected_df.head(5)

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
647,J,"Khan, J; Jain, IP",NaN,NaN,NaN,"Khan, Jameel; Jain, I. P.",NaN,NaN,Chloride catalytic effect on hydrogen desorpti...,INTERNATIONAL JOURNAL OF HYDROGEN ENERGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000376695800059,0
102,J,"Yu, HG; Yang, X; Jiang, XJ; Wu, YM; Chen, SP; ...",NaN,NaN,NaN,"Yu, Hongen; Yang, Xue; Jiang, Xiaojing; Wu, Yi...",NaN,NaN,LaNi5.5 particles for reversible hydrogen stor...,NANO ENERGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000618009300001,0
306,J,"McGrath, AJ; Wadge, MD; Adams, M; Manickam, K;...",NaN,NaN,NaN,"McGrath, Alexander J.; Wadge, Matthew D.; Adam...",NaN,NaN,Stoichiometry and annealing condition on hydro...,INTERNATIONAL JOURNAL OF HYDROGEN ENERGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:001139418300001,0
810,J,"He, SX; Li, GX; Wang, Y; Liu, L; Lu, ZQ; Xu, L...",NaN,NaN,NaN,"He, Shixuan; Li, Guangxu; Wang, Ye; Liu, Liu; ...",NaN,NaN,Achieving both high hydrogen capacity and low ...,INTERNATIONAL JOURNAL OF HYDROGEN ENERGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000921198700001,0
507,J,"Sun, F; Yan, MY; Ye, JH; Liu, XP; Jiang, LJ",NaN,NaN,NaN,"Sun, Fei; Yan, Min-yan; Ye, Jian-hua; Liu, Xia...",NaN,NaN,Effect of CO on hydrogen storage performance o...,JOURNAL OF ALLOYS AND COMPOUNDS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WOS:000342654000007,0


In [ ]:
#convert the abstracts into a list
text_inputs = selected_df['Abstract'].tolist()

In [ ]:
text_inputs[0]

'Hydrogen is considered as an environmental friendly energy carrier because it generates no pollution. Chlorides are promising catalyst for the hydrogen desorption in sodium alanate. All reactions and operations were performed under argon atmosphere in glove box. Crystalline NaAlH4 and catalyst TiCl3, Nb2O5 PbCl2 and CeCl3 were mixed in a ratio of (1, 2) wt%. Due to the air sensitivity of the samples they were covered with a kapton tape layer to minimize any contamination for characterization purpose. Structural characterizations and phase compositions of the samples were studied by XRD. The hydrogen storage capacity is measured by seivert type apparatus. Amount of desorbed hydrogen was found to be 3.6 wt % to 5.5 wt% at 250 degrees C with different type of catalyst. In the present work effect of doping of TiCl3, Nb2O5 PbCl2 and CeCl3 catalysis in NaAlH4 are investigated to understand the hydrogen desorption kinetics. It is concluded that enhanced storage capacity of NaAlH4 can be achi

In [ ]:
#input here the columns you need
text_instruct = """
  Describe all the parameters of the material discussed in the text.
  If no information is available just write "N/A".
  The output should be concise and in the format as below:

  Name of Alloy                        :
  Hydrogen storage capacity            :
  Temperature                          :
  Pressure                             :
  Experimental Conditions              :
  """

In [ ]:
#Generate the output for the abstracts
generated_outputs = []
for i in range(0,30):
    prompt = alpaca_prompt.format(
        text_instruct,  # instruction
        text_inputs[i], # input
        ""              # output
    )
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_outputs.append(decoded_output[0])

In [ ]:
with open("generated_outputs.txt", "w") as f:
  for output in generated_outputs:
    f.write(output + "\n")

In [ ]:
#split the outputs
import pandas as pd
import re

def extract_parameters(text):
    patterns = {
        'Name of Alloy': r'Name of Alloy\s*:\s*(.+)',
        'Hydrogen storage capacity': r'Hydrogen storage capacity\s*:\s*(.+)',
        'Temperature': r'Temperature\s*:\s*(.+)',
        'Pressure': r'Pressure\s*:\s*(.+)',
        'Experimental Conditions': r'Experimental Conditions\s*:\s*(.+)'
    }

    results = {}
    for param, pattern in patterns.items():
        match = re.search(pattern, text)
        results[param] = match.group(1) if match else 'N/A'

    return results

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

file_path = 'generated_outputs.txt'
file_contents = read_file(file_path)

# Split into blocks, ignore first empty part
blocks = re.split(r'###\s*Instruction:', file_contents)[1:]

data = []
for block in blocks:
    parts = re.split(r'###\s*(Input|Response):', block)
    if len(parts) >= 3:  # Ensure we have both Input and Response
        input_text = parts[2].strip()
        response_text = parts[4].strip() if len(parts) >= 5 else ''

        parameters = extract_parameters(response_text)
        parameters['Input'] = input_text

        data.append(parameters)

result_df = pd.DataFrame(data)

if 'Input' in result_df.columns:
    columns = ['Input'] + [col for col in result_df.columns if col != 'Input']
    result_df = result_df[columns]

In [ ]:
doi_values = selected_df['DOI'].tolist()[:30]
result_df['DOI'] = doi_values
result_df

,Input,Name of Alloy,Hydrogen storage capacity,Temperature,Pressure,Experimental Conditions,DOI
0,Hydrogen is considered as an environmental fri...,NaAlH4,3.6 wt% to 5.5 wt%,250 degrees C,N/A,N/A,10.1016/j.ijhydene.2015.11.116
1,"In this work, we demonstrate that the classic ...",LaNi5,5.5 wt% H-2 (based on 12 HNEC),453 K,N/A,N/A,10.1016/j.nanoen.2020.105476
2,This study presents the effect of stoichiometr...,AB2-type hydrogen storage alloys,1.5 wt%,253 K,9.4 bar,Pressure composition isotherm (PCI) measurements,10.1016/j.ijhydene.2023.12.062
3,AlH3 is a metastable hydride with a high hydro...,Hydrogen storage capacity :,Temperature :,Pressure :,Experimental Conditions :,"Below is an instruction that describes a task,...",10.1016/j.ijhydene.2022.10.198
4,(2LiNH(2) + MgH2) material is one of the most ...,(2LiNH(2) + MgH2),4.73 wt.% - 3.88 wt.%,100 degree celsius,2.0 MPa,hydrogen containing I mol% CO as the hydrogena...,10.1016/j.jallcom.2014.07.128
5,"In the present work, two new reversible hydrog...",NaBH4 + LaF3,3.0 wt% at 238 degrees C,238 degrees C,N/A,N/A,10.1039/c4ta00715h
6,"The structure, hydrogen storage and electroche...","La-alloy, Nd-alloy and Cr-alloy",1.23 wt%H-2,N/A,N/A,N/A,10.1016/j.ijhydene.2020.12.071
7,"In this work, the electronic properties of hyd...","LaNi5/A (A = 10 wt.% C, Cu, Pd, Ni) materials ...",Temperature:,Pressure:,Experimental Conditions: XPS valence bands mea...,XPS valence bands measured for mechanically al...,10.1016/j.ijhydene.2011.04.039
8,The development of efficient and reliable ener...,LaN43Co0.4Al0.3,4.3 wt%,25C,35 wt% KOH,Sieverts type of apparatus produced by Labtech...,10.1016/j.jpowsour.2008.01.044
9,Transition metal catalysts are particularly ef...,K2MoO4,5.5 wt%,200 degrees C,3 MPa,"Temperature rise hydrogen absorption test, at ...",10.3390/nano12142468
